In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [2]:
df_huffpo = pd.read_parquet("../../data/raw/news_categories.parquet", columns=["link", "category"])

In [3]:
df_huffpo["slug"] = df_huffpo.link.apply(lambda x: x.split("entry/")[-1].split("_")[0].replace("-", " "))

In [4]:
pcts = (df_huffpo.groupby("category").count() / len(df_huffpo)) * 100
df_huffpo = df_huffpo[df_huffpo.category.isin(pcts[pcts.link >= 2].index)]

In [5]:
le = LabelEncoder()
df_huffpo['label_encoded'] = le.fit_transform(df_huffpo['category'])

In [6]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df_huffpo['slug'], df_huffpo['label_encoded'], test_size=0.3)
val_texts, test_texts, val_labels, test_labels = train_test_split(val_texts, val_labels, test_size=0.33)

In [7]:
# Load tokenizer and model
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(le.classes_))

/Users/nrh146/.pyenv/versions/3.12.5/envs/url-clf/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Tokenize data
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

In [9]:
# Prepare datasets
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels})

In [10]:
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 28978/28978 [00:02<00:00, 12503.46 examples/s]


In [24]:
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})
tokenized_test = test_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 14273/14273 [00:01<00:00, 12835.39 examples/s]


In [15]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [19]:
device = torch.device("mps")
model = model.to(device)

In [21]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val
)

In [22]:
# Train the model
trainer.train()

  0%|          | 10/18924 [00:18<9:21:47,  1.78s/it]

{'loss': 2.6709, 'grad_norm': 1.8923208713531494, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 20/18924 [00:36<9:24:20,  1.79s/it]

{'loss': 2.6637, 'grad_norm': 2.2499585151672363, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


  0%|          | 30/18924 [00:54<9:19:16,  1.78s/it]

...

100%|█████████▉| 18920/18924 [10:01:41<00:07,  1.84s/it]

{'loss': 0.3929, 'grad_norm': 12.712198257446289, 'learning_rate': 1.0855405992184108e-08, 'epoch': 3.0}


100%|██████████| 18924/18924 [10:01:48<00:00,  1.91s/it]

{'train_runtime': 36108.6525, 'train_samples_per_second': 8.384, 'train_steps_per_second': 0.524, 'train_loss': 0.7197749201287883, 'epoch': 3.0}


TrainOutput(global_step=18924, training_loss=0.7197749201287883, metrics={'train_runtime': 36108.6525, 'train_samples_per_second': 8.384, 'train_steps_per_second': 0.524, 'total_flos': 4.011282243455386e+16, 'train_loss': 0.7197749201287883, 'epoch': 3.0})

In [23]:
# Save the model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')

# Save label encoder
import joblib
joblib.dump(le, './fine_tuned_model/label_encoder.joblib')

print("Fine-tuning complete. Model and tokenizer saved.")

Fine-tuning complete. Model and tokenizer saved.


In [26]:
# Run inference on test set
from sklearn.metrics import accuracy_score

predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=1)
accuracy = accuracy_score(test_labels, preds)
print(f"Accuracy on test set: {accuracy}")

100%|██████████| 224/224 [08:57<00:00,  2.40s/it]

Accuracy on test set: 0.7516289497652912


In [29]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(test_labels, preds, average='micro')
print(f"Precision on test set: {precision}")

recall = recall_score(test_labels, preds, average='micro')
print(f"Recall on test set: {recall}")

f1 = f1_score(test_labels, preds, average='micro')
print(f"F1 on test set: {f1}")

Precision on test set: 0.7516289497652912
Recall on test set: 0.7516289497652912
F1 on test set: 0.7516289497652912
